In [3]:
import os
import re
import pandas as pd

# Analiza resultados generales

In [61]:
def select_files(files, name="version 3 drought 2_5"):
    files = [f for f in files if name in f]
    return files


def count_significative_vars(s):
    
    s = s.str.strip()
    is_p01  = s.str.contains("*", regex=False)
    is_p005 = s.str.contains("**", regex=False)
    is_p001 = s.str.contains("***", regex=False)
    
    is_negative  = s.str.contains("-.*\.", regex=True)

    count_positive_p01  = (is_p01 & ~is_negative).sum()
    count_positive_p005 = (is_p005 & ~is_negative).sum()
    count_positive_p001 = (is_p001 & ~is_negative).sum()

    count_negative_p01  = (is_p01 & is_negative).sum()
    count_negative_p005 = (is_p005 & is_negative).sum()
    count_negative_p001 = (is_p001 & is_negative).sum()
    
    count_estimates = s[s != ""].count() 
    return count_estimates, count_positive_p01, count_positive_p005, count_positive_p001, count_negative_p01, count_negative_p005, count_negative_p001

def dict_to_df(dict):
    results_df = pd.DataFrame().from_dict(dict, orient="index", columns=["estimates", "p=0.1 (+)", "p=0.05 (+)", "p=0.01 (+)", "p=0.1 (-)", "p=0.05 (-)", "p=0.01 (-)"]).reset_index(names="Specification")
    return results_df

def process_significance(s):
    # Replace all values that does not contain an asterisk with NaN and turn them into floats
    s = s.where(s.str.contains("*", regex=False))
    return s

def make_float(s):
    # Remove asterisks and turn into floats
    s = s.replace(r'[^0-9\-.]', "", regex=True)
    return s

def parse_name(name):
    print(name)
    conditions = name.split(" - ")

    # # From 2003 or Last 10 years
    # if ("from 2003" in conditions[-1]) or ("last 10 years" in conditions[-1]):
    #     subset = conditions[-1].replace(".tex", "")
    #     conditions = conditions[:-1]
    # elsC:
    subset = "Last 10 years & from 2003"
    
    # Control variables
    if "controls" in conditions[-1]:
        controls = conditions[-1].replace(".tex", "")
        conditions = conditions[:-1]
    elsC:
        controls = "Standard Controls"
    
    # Fixed Effects
    if any([kword in conditions[-1] for kword in ["Decade", "No Trend", "Country FE"]]):
        fixed_effects = conditions[-1].replace(".tex", "")
        conditions = conditions[:-1]
    elsC:
        fixed_effects = "Standard FE"
    
    assert len(conditions)==1
    
    words = conditions[0]
    words = words.strip().split(" ")
    
    version_nam, version_num = words[:2]

    if version_num in ["1", "2"]:
        months_num = words[2]
        agg_timeframe = words[3]
        temp_variable = words[4]
        drought_num = " - "

    elif version_num=="3":
        months_num = words[3]
        agg_timeframe = "min-max"
        temp_variable = words[5]
        drought_num = words[4]
        drought_num = drought_num.replace("_", ".") + " SD"

    elif version_num=="4":
        months_num = words[3]
        agg_timeframe = words[6]
        temp_variable = words[5]
        drought_num = words[4]
        drought_num = drought_num.replace("_", ".") + " SD"

    out_dict = {
        "Version": version_num, 
        "Variable Treshold": drought_num, 
        "SPI Timeframe": months_num,
        "Agreggation": agg_timeframe,
        "Temperature": temp_variable,
        "Fixed Effects": fixed_effects,
        "Controls set": controls,
        "Subset": subset
    }
    return out_dict

In [62]:
i=0
for name in all_results_df.file.sample(10).values:
    print(parse_name(name))

version 4 drought SPI12 2_5 avg std_t  - controls1.tex
['version 4 drought SPI12 2_5 avg std_t ']
4
{'Version': '4', 'Variable Treshold': '2.5 SD', 'SPI Timeframe': 'SPI12', 'Agreggation': 'std_t', 'Temperature': 'avg', 'Fixed Effects': 'Standard FE', 'Controls set': 'controls1', 'Subset': 'Last 10 years & from 2003'}
version 4 drought SPI1 1_5 min-max stdm_t  - controls2.tex
['version 4 drought SPI1 1_5 min-max stdm_t ']
4
{'Version': '4', 'Variable Treshold': '1.5 SD', 'SPI Timeframe': 'SPI1', 'Agreggation': 'stdm_t', 'Temperature': 'min-max', 'Fixed Effects': 'Standard FE', 'Controls set': 'controls2', 'Subset': 'Last 10 years & from 2003'}
version 3 drought SPI6 2_0 min-max  - controls1.tex
['version 3 drought SPI6 2_0 min-max ']
3
{'Version': '3', 'Variable Treshold': '2.0 SD', 'SPI Timeframe': 'SPI6', 'Agreggation': 'min-max', 'Temperature': 'min-max', 'Fixed Effects': 'Standard FE', 'Controls set': 'controls1', 'Subset': 'Last 10 years & from 2003'}
version 1 SPI12 min-max t  - 

# Genera tabla de regresiones

In [5]:
import re
from tqdm import tqdm

folder = r"C:\Working Papers\Paper - Child mortality and Climate Shocks\Outputs"
files = os.listdir(folder)
files = [f for f in files if ".tex" in f]
assert len(files)!=0


for i, file in tqdm(enumerate(files), total=len(files)):

    filename = rf"{folder}\{file}"
    df = pd.read_csv(filename,
                    sep='&',
                    header=0,
                    skiprows=4,
                    skipfooter=3,
                    engine='python')

    keywords = "drought|excessiverain|t_|std_t|stdm_t|spi" # Regex allowed
    
    df = df.set_index(df.columns[0]) # Set variable names in index
    df = df[df.index != ' '] # Remove standard errors
    df = df[df.index.str.contains(f'{keywords}|VARIABLES')]

    results = {}
    for specification in df.columns:
        counts = count_significative_vars(df[specification])
        specification = re.sub(r'[^0-9a-zA-Z]+', '', specification)
        results[specification] = counts

    results_df = dict_to_df(results)    
    results_df["path"] = filename
    results_df["file"] = file
    results_df = results_df[['file', 'path', 'Specification', 'estimates', 'p=0.1 (+)', 'p=0.05 (+)', 'p=0.01 (+)', 'p=0.1 (-)', 'p=0.05 (-)', 'p=0.01 (-)']]

    if i==0:
        results_df.to_csv("regs_significance_all_models.csv", index=False)    
    elsC:
        results_df.to_csv("regs_significance_all_models.csv", mode="a", index=False, header=False)
        
# ### Process data
all_results_df = pd.read_csv("regs_significance_all_models.csv")

# Consistency in specification
all_results_df["Consistent (p=0.1)"]  = (all_results_df["p=0.1 (+)"]  >= 0) & (all_results_df["p=0.1 (-)"]  == 0)
all_results_df["Consistent (p=0.05)"] = (all_results_df["p=0.05 (+)"] >= 0) & (all_results_df["p=0.05 (-)"] == 0)
all_results_df["Consistent (p=0.01)"] = (all_results_df["p=0.01 (+)"] >= 0) & (all_results_df["p=0.01 (-)"] == 0)

# Consistency in regression set
regressions_set = all_results_df.groupby("path")
for pval in ["0.1 (+)", "0.05 (+)", "0.01 (+)", "0.1 (-)", "0.05 (-)", "0.01 (-)"]:
    all_results_df[f"Set {pval}"] = regressions_set[f"p={pval}"].transform("sum")
    all_results_df[f"Set estimates"] = regressions_set[f"estimates"].transform("sum")
    all_results_df[f"Set {pval} (%)"] = all_results_df[f"Set {pval}"] / all_results_df[f"Set estimates"]
       
# Parse the names of the
all_results_df = all_results_df.file.apply(parse_name).apply(pd.Series).join(all_results_df)

all_results_df.to_excel("regs_significance_all_models.xlsx", index=False)    
os.remove("regs_significance_all_models.csv")
print("Done! File saved as regs_significance_all_models.xlsx")

100%|██████████| 432/432 [00:12<00:00, 35.35it/s]


AssertionError: 

In [14]:
parse_name(results_df.file.values[0])

('version', '4', 'drought', '2_5', '6', 'months.tex')

In [5]:
df.columns

Index(['        (1) ', '        (2) ', '        (3) ', '        (4) ',
       '       (5) ', '       (6) ', '       (7) ', '       (8) ',
       '       (9) ', '      (10) ', '      (11) ', '      (12) ',
       '      (13) ', '      (14) ', '      (15) ', '      (16) \\ '],
      dtype='object')

In [12]:
df['        (1) '].str.strip() == ""

                                            
drought6\_2\_5\_q1                              False
drought6\_2\_5\_q2                              False
drought6\_2\_5\_q3                              False
drought6\_2\_5\_30d                             False
excessiverain6\_2\_5\_q1                        False
excessiverain6\_2\_5\_q2                        False
excessiverain6\_2\_5\_q3                        False
excessiverain6\_2\_5\_30d                       False
drought6\_2\_5\_30d3m                            True
excessiverain6\_2\_5\_30d3m                      True
drought6\_2\_5\_3m6m                             True
excessiverain6\_2\_5\_3m6m                       True
drought6\_2\_5\_6m12m                            True
excessiverain6\_2\_5\_6m12m                      True
Name:         (1) , dtype: bool

# Resultados por país

In [78]:
import plotly.graph_objects as go

def dot_plot(all_specs, varname, quarter):
    
    fig = go.Figure()

    colors = [
        'rgba(156, 165, 196, 0.95)','rgba(156, 165, 196, 0.95)','rgba(156, 165, 196, 0.95)','rgba(156, 165, 196, 0.95)',
        'rgba(204, 204, 204, 0.95)','rgba(204, 204, 204, 0.95)','rgba(204, 204, 204, 0.95)','rgba(204, 204, 204, 0.95)',
        'rgba(255, 166, 86, 0.95)','rgba(255, 166, 86, 0.95)','rgba(255, 166, 86, 0.95)','rgba(255, 166, 86, 0.95)',
        'rgba(240, 230, 140, 0.95)','rgba(240, 230, 140, 0.95)','rgba(240, 230, 140, 0.95)','rgba(240, 230, 140, 0.95)',
    ]
    data = all_specs[
        all_specs.variable.str.contains("drought") &
        all_specs.variable.str.contains(quarter)
    ]
    for i, spec in enumerate(data.specification.unique()):
        data_plot = data[data.specification == spec]
        fig.add_trace(go.Scatter(
            x=data_plot["value"].astype("float"),
            y=data_plot["Country"],
            marker=dict(
                color=colors[i],
                size=16
            ),
            mode='markers',
            name=spec,
        ))

    fig.update_traces(mode='markers', marker=dict(line_width=1, symbol='circle', size=16))

    fig.update_layout(
        title=varname,
        xaxis=dict(
            showgrid=False,
            showline=True,
            linecolor='rgb(102, 102, 102)',
            tickfont_color='rgb(102, 102, 102)',
            showticklabels=True,
            ticks='outside',
            tickcolor='rgb(102, 102, 102)',
        ),
        margin=dict(l=140, r=40, b=50, t=80),
        legend=dict(
            font_size=10,
            yanchor='top',
            xanchor='center',
        ),
        width=1600,
        height=600,
        paper_bgcolor='white',
        plot_bgcolor='white',
        hovermode='closest',
    )
    
    # Add a vertical line at zero
    fig.update_layout(shapes=[
        dict(
            type="line",
            x0=0,
            y0=0,
            x1=0,
            y1=all_specs.Country.drop_duplicates().shape[0],
            line=dict(
                color="red",
                width=2,
                dash="dashdot",
            )
        )
    ])

    fig.show()

In [81]:
import re
from tqdm import tqdm

folder = r"Z:\Laboral\Working Papers\Paper - Child mortality and Climate Shocks\Outputs\countries"
name = "version 3 drought 2_5 12 months"
version_nam, version_num, drought_nam, drought_num, months_num, months_nam = parse_name(name)

files = os.listdir(folder)
files = [f for f in files if ".tex" in f]
files = select_files(files, name=name)

all_specs = []
for file in tqdm(files):

    filename = rf"{folder}\{file}"
    country = re.search(r"([A-Z]{3})", file).group(0)
    
    df = pd.read_csv(filename,
                    sep='&',
                    header=0,
                    skiprows=4,
                    skipfooter=3,
                    engine='python')

    keywords = "temp|spi|drought|excessiverain|prec" # Regex allowed
    
    # Process data
    df = df.set_index(df.columns[0]) # Set variable names in index
    df = df[df.index != ' '] # Remove standard errors
    df = df[df.index.str.contains(keywords)]
    df.index = df.index.str.strip()
    df.index.name = "variable"
    df = df.reset_index()
    results = {}
    df.columns = [re.sub(r'[^0-9a-zA-Z]+', '', col) for col in df.columns]

    # Format dataframe
    drought_num_str = drought_num.replace("_", r"\_")
    months_num_str = fr"\_{months_num}\_"
    country_specs = df.melt(id_vars=["variable"], var_name="specification", value_name="value")
    country_specs["value"] = process_significance(country_specs.value)    
    country_specs["value"] = make_float(country_specs.value)    
    country_specs["Country"] = country
    country_specs["variable"] = country_specs["variable"].str.replace(months_num_str, " ").str.replace(drought_num_str, drought_num)
    all_specs += [country_specs]
    
all_specs = pd.concat(all_specs)

dot_plot(all_specs, f"drought", "q1")
dot_plot(all_specs, f"drought", "q2")
dot_plot(all_specs, f"drought", "q3")
dot_plot(all_specs, f"drought", "30d")
dot_plot(all_specs, f"drought", "30d3m")
dot_plot(all_specs, f"drought", "3m6m")
dot_plot(all_specs, f"drought", "6m12m")

 17%|█▋        | 9/54 [00:00<00:00, 83.24it/s]C:\Users\ofici\AppData\Local\Temp\ipykernel_24444\3964789092.py:30: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\ofici\AppData\Local\Temp\ipykernel_24444\3964789092.py:30: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

 54%|█████▎    | 29/54 [00:00<00:00, 85.37it/s]C:\Users\ofici\AppData\Local\Temp\ipykernel_24444\3964789092.py:30: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly cal